# Package Installation

The below command installs the required Python packages for using LangChain, OpenAI, ChromaDB, and Google Search Results.

In [ ]:
!pip install langchain openai chromadb google-search-results

# Setting API Keys

Here, we are setting environment variables for the OpenAI and SerpAPI keys to authenticate API requests.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-tRy9vuNQVlSRaPcBVTqVT3BlbkFJ*********************"
os.environ["SERPAPI_API_KEY"] = "2ca48c0eaad95d0b9c8d9b405773cba672f92be45c2178071*********************"

# AutoGPT

We are now importing specific modules and classes required for LangChain, including tools for interacting with OpenAI, SerpAPI, and file management.

In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool

from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

# Tool Configuration

This section configures various tools that the agent will use, including a search tool powered by SerpAPI, WriteFileTool, and ReadFileTool for file management.

In [ ]:
search = SerpAPIWrapper(serpapi_api_key=os.environ["SERPAPI_API_KEY"])
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

# Vector Store and Embeddings

This part initializes the vector store using FAISS, a high-performance similarity search library, and sets up embeddings using OpenAI's model.

In [ ]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

# Agent

Here, an AutoGPT agent is initialized using the configured tools, an LLM model (ChatOpenAI), and the defined memory store (vectorstore).

In [ ]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Vaibhav_AutoGPT",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

# Task Execution by Agent

The agent is finally set to run with a specific prompt ("Top news of Indian stock market today").

In [ ]:
agent.run(["Top news of Indian stock market today"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Vaibhav_AutoGPT, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Top news of Indian stock market today


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args 

'I have completed all my tasks.'

# Reading the prepared news details

In [ ]:
f = open('/content/indian_stock_market_highlights.txt', 'r')
print(f.read())

The Indian stock market today saw gains in the banking and IT sectors, with the Nifty Bank and Nifty IT indices rising by 1.5% and 2.0% respectively. However, the overall market was dragged down by losses in the energy and metal sectors, with the Nifty Energy and Nifty Metal indices falling by 0.5% and 1.0% respectively.


In [ ]:
f.close()